## Consistency

- Consistency is the property of a transaction 
- that guarantees that the changes done will be following all the defined rules.

### Database Transaction

- a single change in database
- a transaction can involve multiple operations
- e.g. money transfer (**transaction**):
1. subtraction from buyer
2. addition to seller
- e.g. item transfer

- money and item transfer could be also one transfer

What is a transaction/operation depends on the nature of the application.

## ACID
- The properties 
1. atomic 
2. consistent
3. isolated
4. durable 
are commonly known as the ACID properties transactions.

1. Setup

In [ ]:
-- Drop the table if it exists
DROP TABLE IF EXISTS bank_accounts;

-- Create the example table with the constraint for non-negative balance
CREATE TABLE bank_accounts (
    account_id SERIAL PRIMARY KEY,
    account_name TEXT,
    account_balance DECIMAL DEFAULT 0 CHECK (account_balance >= 0)  -- Constraint added here
);

-- Insert sample data for the exercises
INSERT INTO bank_accounts (account_name, account_balance) VALUES ('A', 100), ('B', 200);


### Atomicity:

A database transaction must be done as a whole or not done at all.

- Ask two students (let's call them A and B) to simultaneously attempt a money transfer operation. However, in student B's query, intentionally create a scenario that would result in a negative balance (violating our constraint).

In [ ]:
BEGIN;
UPDATE bank_accounts SET account_balance = account_balance - 50 WHERE account_name = 'A';
UPDATE bank_accounts SET account_balance = account_balance + 50 WHERE account_name = 'B';
COMMIT;


In [ ]:
BEGIN;
UPDATE bank_accounts SET account_balance = account_balance - 150 WHERE account_name = 'A'; -- This will violate our non-negative constraint
UPDATE bank_accounts SET account_balance = account_balance + 150 WHERE account_name = 'B';
COMMIT; --will not work
ROLLBACK; -- Due to the constraint violation above


Observation: The constraint violation in Student B's transaction should cause the entire transaction to roll back, demonstrating Atomicity.

### Consistency
- Data is in a consistent state when a transaction starts and when it ends.
e.g.an application that transfers funds from one account to another:
the consistency property ensures that

the *total value of funds in both the accounts* is the same at the start and end of each transaction.


In [ ]:
SELECT sum(account_balance) FROM bank_accounts;

### Isolation

- A database transaction must also be isolated.
- A higher level of isolation has higher chances of one transaction blocking another.
- A lower level of isolation has higher chances of producing concurrency undesired effects.

Instructions:
- Ask two students (let's call them C and D) to begin a transaction simultaneously. Student C will update an account balance, but will delay the commit. Student D will try to read the same account.

Student C:
```sql
BEGIN;
UPDATE bank_accounts SET account_balance = account_balance + 100 WHERE account_name = 'C';
-- Ask the student to wait before committing
```

Student D (run this immediately after student C runs their update):
```sql
SELECT account_balance FROM bank_accounts WHERE account_name = 'C';
```

After a few seconds, Student C:
```sql
COMMIT;
```

Now, Student D:
```sql
SELECT account_balance FROM bank_accounts WHERE account_name = 'C';
```

**Observation**: Student D will first see the old balance and then the updated balance after Student C commits the transaction. This demonstrates Isolation.

### Durability
- A database transaction must also be durable.
- The changes should be persistent.
- Once a change is done, it is written on the file system
- only another change will remove that change.

### ACID COMPLIANCE
- postgres
- mysql
- sqlite

All Relational DBMS are, or can be, ACID compliant.

Some NoSQL DBMS can be ACID compliant:

- MongoDB
- CouchDB
- Db2

But ACID is a relational concept and most NoSQL use the BASE properties
- BASE properties focus on availability rather than consistency


### BASE Model

The BASE model is an alternative transaction model to the traditional ACID model.

The need for an alternative arose from the challenges faced when trying to achieve **high availability, fault tolerance, and scalability** in distributed and NoSQL databases. 

BASE emphasizes a looser, more scalable approach, which is a fit for many modern web and cloud-based applications.

Here's a breakdown of the BASE acronym:

1. **Basically Available**: 
    - This suggests that the system does guarantee availability, but might not guarantee up-to-the-moment data accuracy across the entire system. 
    - Data might be stale or might not be synchronized across all nodes or replicas. 
    - In practice, this means that the system continues to function even if a portion of its infrastructure is offline or has failed.

2. **Soft State**: 
    - The state of the system might change over time, even without any input. This is due to the eventual consistency model.
    - The "soft" nature indicates that the system's state may not remain consistent at all times. As opposed to "hard state" systems, where a change leads to an immediate and consistent new state everywhere, "soft state" systems recognize and allow temporary inconsistencies.

3. **Eventually Consistent**: 
    - While the system may not always reflect a consistent state to all users at all times, it promises that, given a certain amount of time without new updates, it will eventually converge towards a consistent state.
    - This contrasts with the immediate consistency seen in ACID systems. 
    - when an update is made in a distributed database, that update will be reflected in all database nodes
    (Horizontal scaling: add nodes)
    (vs. vertical scaling: increase the resources of a single machine) 

The trade-offs between ACID and BASE depend on the specific use case. 

- **ACID** guarantees strict consistency, atomicity, and isolation, making it a fit for systems where correctness and consistency are paramount (e.g., banking transactions).
  
- **BASE**, on the other hand, trades consistency for availability and fault tolerance. It is more suitable for systems where high availability is essential, and where the application can tolerate temporary inconsistencies (e.g., social media posts, caching systems).

In real-world scenarios, many systems don't strictly adhere to just ACID or BASE but find a balance between them, depending on the specific requirements of various parts of the system.

#### Eventually Consistent
![](event_consistent.png)


ACID versus BASE

- NoSQL databases provide so-called BASE guarantees
- sacrifice a degree of consistency in order to increase availability
- Rather than providing strong consistency, they provide eventual consistency
- a datastore that provides BASE guarantees can occasionally fail to return the result of the latest WRITE

In a typical example, an eventual consistency NoSQL database might permit a hotel room to be reserved on the same night by two different customers, one in Hong Kong and the other in New York City.

For many business applications, this is not a problem.

The resilience, availability, and low-latency afforded by NoSQL are well worth the tradeoff. 

The hotel can provide perks to a disgruntled customer, but, by contrast, a regional outage could create significant churn.




## Database Concurrency
- is the ability to execute two or more transactions at the same time.




### READ vs. write
- Reading transactions can be executed simultaneously.
- Writing transactions very often cannot be executed simultaneously.

The nature of the transactions determines how the concurrent transactions are managed.

### Concurrency Issues
The isolation property prevents the occurrence of some undesired behavior due to concurrent writing transactions.

1. Dirty reads
- A transaction reads data written by concurrent uncommitted transactions.
- we are not sure about the consistency of the data that is read because we don’t know the result of the open transaction
2. Non-repeatable reads
- A transaction re-reads data it has previously read and finds that data has been modified by another transaction
- occurs, when during the course of a transaction, a row is retrieved twice and the values within the row differ between reads.
- user b commits between two queries of user a
3. Phantom reads
- the result changes if executed again, due to another recently-committed transaction.
- All the rows in the query have the same value before and after, but different rows are being selected (because B has deleted or inserted some)
- when reading from committed INSERTS and/or DELETES from another transaction.


### Concurrent Writing Transactions

- Concurrent writing transactions are executed sequentially to prevent isolation issues.
- While the system executes the first transaction the rest stays “on hold”, waiting for the first to finish.

- So how do Write Transaction 2 & 3 know they need to wait?
1. A writing transaction request arrives. It checks if it has access to the data. It does, so it executes.
2. When the transaction is executed, a **lock** is added to the database.
3. A second transaction request arrives. It checks if it has access to the data. It finds the lock, so it waits.
4. When the first transaction finishes, the lock is released and the next transaction checks again.
5. A COMMIT; and a ROLLBACK; will release the lock.


### Some writing transactions can be done simultaneously.

- Each new transaction has a lock and a key. 
- Checks if the key fits in the current lock.
- If the key fits, adds its own lock and executes.
- A third transaction will try to open every lock it finds on its way to the data. 
- If it succeeds, adds a new lock and executes.

In [ ]:
psql -h host_name_or_ip -p port_number -U myuser mydb

### Locks in PostgreSQL
In databases, a lock is a mechanism employed to control access to a database resource by multiple transactions. 

The primary objective of locks is to ensure data integrity in concurrent access scenarios.

By using locks, PostgreSQL ensures that two transactions don't interfere with each other,
especially in scenarios that could compromise the integrity of the database.

- to see thr locks, run:
```sql
SELECT relation::regclass, mode, granted 
FROM pg_locks 
WHERE relation = 'my_table'::regclass;
```


1. try to do a select transaction (sessionA) and a drop transaction(session B)

2. then do select (A) and insert (B) (non conflicting locks)

3. create index (A) and insert (B) conflicting



1. **ACCESS SHARE**
   - **Typically Acquired By**: `SELECT ...`
   - **Conflicts with**: ACCESS EXCLUSIVE
   - **SQL Example**:
     ```sql
     SELECT * FROM my_table;
     ```

2. **ROW EXCLUSIVE**
   - **Typically Acquired By**: `INSERT`, `UPDATE`, `DELETE`
   - **Conflicts with**: SHARE, SHARE ROW EXCLUSIVE, EXCLUSIVE, ACCESS EXCLUSIVE
   - **SQL Example**:
     ```sql
     INSERT INTO my_table VALUES (1, 'data');
     UPDATE my_table SET column_name = 'new_value' WHERE condition;
     DELETE FROM my_table WHERE condition;
     ```

3. **SHARE**
   - **Typically Acquired By**: `CREATE INDEX` (not concurrently)
   - **Conflicts with**: ROW EXCLUSIVE, SHARE ROW EXCLUSIVE, EXCLUSIVE, ACCESS EXCLUSIVE
   - **SQL Example**:
     ```sql
     CREATE INDEX idx_column_name ON my_table(column_name);
     ```

4. **ACCESS EXCLUSIVE**
   - **Typically Acquired By**: `ALTER TABLE`, `DROP TABLE`, etc.
   - **Conflicts with**: All other locks (ACCESS SHARE, ROW SHARE, ROW EXCLUSIVE, SHARE UPDATE EXCLUSIVE, SHARE, SHARE ROW EXCLUSIVE, EXCLUSIVE)
   - **SQL Example**:
     ```sql
     ALTER TABLE my_table ADD COLUMN new_column INT;
     DROP TABLE my_table;
     ```
